# Parse/Clean NJSP Fatal Crash XMLs
- Load XMLs
- Clean / Assign some dtypes
- Write to parquet and SQLite

In [1]:
from utz import *
from nj_crashes.paths import RUNDATE_PATH
from nj_crashes.utils import parse_file

### Load+Parse NJSP fatal crash data (2008-2023)

In [2]:
cur_month = to_dt(now().strftime('%Y-%m'))
cur_year = cur_month.year

In [3]:
parsed_files = {
    year: parse_file(f'data/FAUQStats{year}.xml')
    for year in range(2008, cur_year + 1)
}
crashes, totals = [
    pd.concat(dfs)
    for dfs in
    zip(*[ 
        [ dfs['crashes'], dfs['totals'] ]
        for dfs in parsed_files.values() 
    ])
]
totals = totals.set_index('year')
totals

540 "COUNTY" entries, 519 containing "MUNICIPALITY"/crash info, 555 accidents, 414 injuries, 590 fatalities
528 "COUNTY" entries, 507 containing "MUNICIPALITY"/crash info, 550 accidents, 352 injuries, 584 fatalities
525 "COUNTY" entries, 504 containing "MUNICIPALITY"/crash info, 530 accidents, 366 injuries, 556 fatalities
575 "COUNTY" entries, 554 containing "MUNICIPALITY"/crash info, 586 accidents, 517 injuries, 627 fatalities
546 "COUNTY" entries, 525 containing "MUNICIPALITY"/crash info, 553 accidents, 382 injuries, 589 fatalities
498 "COUNTY" entries, 477 containing "MUNICIPALITY"/crash info, 508 accidents, 393 injuries, 542 fatalities
517 "COUNTY" entries, 496 containing "MUNICIPALITY"/crash info, 523 accidents, 345 injuries, 556 fatalities
517 "COUNTY" entries, 496 containing "MUNICIPALITY"/crash info, 522 accidents, 374 injuries, 562 fatalities
550 "COUNTY" entries, 529 containing "MUNICIPALITY"/crash info, 570 accidents, 398 injuries, 602 fatalities
580 "COUNTY" entries, 559 co

,accidents,injuries,fatalities
year,,,
2008,555,414,590
2009,550,352,584
2010,530,366,556
2011,586,517,627
2012,553,382,589
2013,508,393,542
2014,523,345,556
2015,522,374,562
2016,570,398,602


In [4]:
rundate = to_dt(parsed_files[cur_year]['rundate'])
rundate_str = rundate.strftime('%Y-%m-%d')
with open(RUNDATE_PATH, 'w') as f:
    json.dump({ 'rundate': str(rundate), }, f)

cur_year_dt = to_dt(str(cur_year)).tz_localize(rundate.tz)
nxt_year_dt = to_dt(str(cur_year + 1)).tz_localize(rundate.tz)
cur_month_dt = cur_month.tz_localize(rundate.tz)

cur_year_dt, cur_month_dt, rundate, nxt_year_dt

/usr/local/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


(Timestamp('2023-01-01 00:00:00'),
 Timestamp('2023-01-01 00:00:00'),
 Timestamp('2023-01-28 10:00:10'),
 Timestamp('2024-01-01 00:00:00'))

In [5]:
crashes['dt'] = crashes[['DATE','TIME']].apply(lambda r: to_dt(f'{r["DATE"]} {r["TIME"]}'), axis=1)
crashes = (
    crashes
    .astype({
        'FATALITIES': float,
        'FATAL_D': float,
        'FATAL_P': float,
        'FATAL_T': float,
        'FATAL_B': float,
        'INJURIES': float,
    })
    .drop(columns=['DATE', 'TIME'])
    .set_index('ACCID')
)
crashes = crashes.sort_values('dt')
crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,INJURIES,STREET,FATAL_D,FATAL_P,FATAL_T,FATAL_B,dt
ACCID,,,,,,,,,,,,,,
1703,01,Atlantic,0102,Atlantic City,446,State/Interstate Authority 446 S MP 1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2008-01-01 00:35:00
1681,09,Hudson,0910,Union City,NaN,Bergenline Ave S MP 0 at 6th St,1.0,NaN,Bergenline Ave,NaN,NaN,NaN,NaN,2008-01-01 04:11:00
1659,04,Camden,0415,Gloucester Twsp,42,State Highway 42 N MP 8.2,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2008-01-01 06:46:00
1661,20,Union,2004,Elizabeth City,624,County 624 W MP 2.2 at Ikea Dr,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2008-01-01 12:29:00
1664,07,Essex,0714,Newark City,95,State/Interstate Authority 95 S MP 104,1.0,0.0,NaN,NaN,NaN,NaN,NaN,2008-01-02 09:09:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12284,20,Union,2013,Rahway City,1,State Highway 1,1.0,NaN,NaN,0.0,0.0,1.0,0.0,2023-01-21 21:31:00
12288,01,Atlantic,0119,Pleasantville City,446,Atlantic City Expressway MP 3,1.0,NaN,NaN,0.0,0.0,1.0,0.0,2023-01-24 17:33:00
12287,13,Monmouth,1334,Neptune Twsp,NaN,2200 RT 66 EB,1.0,NaN,2200 RT 66 EB,0.0,0.0,0.0,1.0,2023-01-24 21:54:00


In [6]:
counties = crashes[['CCODE', 'CNAME']].value_counts().rename('accidents').reset_index().set_index('CCODE').sort_index().CNAME
counties

CCODE
01      Atlantic
02        Bergen
03    Burlington
04        Camden
05      Cape May
06    Cumberland
07         Essex
08    Gloucester
09        Hudson
10     Hunterdon
11        Mercer
12     Middlesex
13      Monmouth
14        Morris
15         Ocean
16       Passaic
17         Salem
18      Somerset
19        Sussex
20         Union
21        Warren
Name: CNAME, dtype: object

In [7]:
munis = (
    crashes
    .groupby('MCODE')
    .apply(
        lambda df: (
            df
            [['MNAME', 'CCODE']]
            .drop_duplicates()
            .set_index('MNAME', drop=True)
        )
    )
    .reset_index(1)
    .sort_index()
)
munis

,MNAME,CCODE
MCODE,,
0101,Absecon City,01
0102,Atlantic City,01
0103,Brigantine City,01
0104,Buena Boro,01
0105,Buena Vista Twsp,01
...,...,...
2119,Phillipsburg To,21
2120,Pohatcong Twsp,21
2121,Washington Boro,21


In [8]:
counties.to_frame().to_parquet('data/counties.pqt')
munis.to_parquet('data/munis.pqt')

In [9]:
muni_counties = crashes.groupby('MNAME').apply(lambda df: df['CNAME'].unique()).rename('counties')
muni_counties

MNAME
Aberdeen Twsp        [Monmouth]
Absecon City         [Atlantic]
Alexandria Twsp     [Hunterdon]
Allamuchy Twsp         [Warren]
Allendale Boro         [Bergen]
                       ...     
Woodland Twsp      [Burlington]
Woodlynne Boro         [Camden]
Woodstown Boro          [Salem]
Woolwich Twsp      [Gloucester]
Wyckoff Twsp           [Bergen]
Name: counties, Length: 497, dtype: object

In [10]:
muni_county_counts = muni_counties.apply(len).rename('muni_county_counts').sort_values()
multi_county_counts = muni_county_counts[muni_county_counts > 1]
multi_county_counts

MNAME
Lawrence Twsp       2
Hamilton Twsp       2
Monroe Twsp         2
Union Twsp          2
Mansfield Twsp      2
Springfield Twsp    2
Hopewell Twsp       2
Ocean Twsp          2
Greenwich Twsp      3
Franklin Twsp       4
Washington Twsp     5
Name: muni_county_counts, dtype: int64

In [11]:
(
    crashes
    .groupby(['CNAME', 'MNAME'])
    .size()
    .rename('accidents')
    .reset_index()
    .merge(
        multi_county_counts,
        left_on='MNAME',
        right_index=True,
    )
    .set_index(['MNAME', 'CNAME'])
    .accidents
)

MNAME             CNAME     
Hamilton Twsp     Atlantic      87
                  Mercer        81
Washington Twsp   Bergen         9
                  Burlington     5
                  Gloucester    42
                  Morris         8
                  Warren        14
Mansfield Twsp    Burlington    19
                  Warren         8
Springfield Twsp  Burlington    14
                  Union         13
Greenwich Twsp    Cumberland     1
                  Gloucester     2
                  Warren         7
Hopewell Twsp     Cumberland    10
                  Mercer        16
Lawrence Twsp     Cumberland     9
                  Mercer        35
Franklin Twsp     Gloucester    67
                  Hunterdon      1
                  Somerset      52
                  Warren         9
Monroe Twsp       Gloucester    88
                  Middlesex     29
Union Twsp        Hunterdon      9
                  Union         51
Ocean Twsp        Monmouth      11
                  Ocean   

### Save to file

In [12]:
from nj_crashes.paths import DB_URI
from sqlalchemy import create_engine

engine = create_engine(DB_URI)

tables = {
    'totals': totals,
    'crashes': crashes,
}

for name, table in tables.items():
    table.to_sql(name, con=engine, if_exists='replace',)
    table.to_parquet(f'data/{name}.pqt')